In [1]:
#Import yaml and various ways to load in the data from the yaml file

import yaml
from yaml.loader import SafeLoader, BaseLoader, FullLoader, UnsafeLoader

In [2]:
#Load in data from yaml file using a safe load. Makes sure to update curr_data when we switch to a new network.

def reset(filepath):
    
    with open(filepath, "r") as stream:
        try:
            data = (yaml.safe_load(stream))
        except yaml.YAMLError as exc:
            print(exc)
    
    return data

In [3]:
#imports for running agent
from itertools import product

import nasim
import pandas as pd
import csv

/Users/abhayraghavan/opt/anaconda3/lib/python3.8/site-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/Users/abhayraghavan/opt/anaconda3/lib/python3.8/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,


In [4]:
#run bruteforce agent
LINE_BREAK = "-"*60


def run_bruteforce_agent(env, cols, step_limit=1e6, verbose=True):
    """Run bruteforce agent on nasim environment.

    Parameters
    ----------
    env : nasim.NASimEnv
        the nasim environment to run agent on
    step_limit : int, optional
        the maximum number of steps to run agent for (default=1e6)
    verbose : bool, optional
        whether to print out progress messages or not (default=True)

    Returns
    -------
    int
        timesteps agent ran for
    float
        the total reward recieved by agent
    bool
        whether the goal was reached or not
    """
    if verbose:
        print(LINE_BREAK)
        print("STARTING EPISODE")
        print(LINE_BREAK)
        print("t: Reward")
        
    env.reset()
    total_reward = 0
    pred_reward = 0
    done = False
    steps = 0
    cycle_complete = False
    col_list = cols
    currData = pd.read_csv("bruteforce_agent.csv", usecols=['Reward', 'Steps', 'Network', 'Change'])

    if env.flat_actions:
        act = 0
    else:
        act_iter = product(*[range(n) for n in env.action_space.nvec])

    while not done and steps < step_limit:
        if env.flat_actions:
            act = (act + 1) % env.action_space.n
            cycle_complete = (steps > 0 and act == 0)
        else:
            try:
                act = next(act_iter)
                cycle_complete = False
            except StopIteration:
                act_iter = product(*[range(n) for n in env.action_space.nvec])
                act = next(act_iter)
                cycle_complete = True

        _, rew, done, _ = env.step(act)
        actiondf = pd.DataFrame({'States': [_],
                                 'Steps': [steps]})
        actiondf.to_csv('states.csv', mode = 'a', index = False, header = False)
        
        total_reward += rew
    
        if cycle_complete and verbose:
            print(f"{steps}: {total_reward}")
        
        stateData = pd.read_csv("states.csv", usecols=['States', 'Steps'])
        #if "'value': 100" in str(stateData["States"][stateData[stateData.columns[0]].count() - 1]):
            #pred_reward += 99.5
        #else:
            #pred_reward -= 1
            
        #print("Predicted reward at step " + str(steps) + ": " + str(pred_reward))      
        print("Total reward at step " + str(steps) + ": " + str(total_reward))
        steps += 1

    #totReward = 0
    #totIter = 0
    #for i in range(0, currData[currData.columns[0]].count()):
        #totReward += currData["Reward"][i]
        #totIter += 1
        
    #predReward = totReward / totIter
    
    if done and verbose:
        print(LINE_BREAK)
        print("EPISODE FINISHED")
        print(LINE_BREAK)
        print(f"Goal reached = {env.goal_reached()}")
        print(f"Total steps = {steps}")
        #print(f"Average reward = {round(predReward)}")
        print(f"Total reward = {total_reward}")
        print(f"Terminating state = {_}")
        stateData = pd.read_csv("states.csv", usecols=['States', 'Steps'])
        print(str(stateData["States"][0]))
    elif verbose:
        print(LINE_BREAK)
        print("STEP LIMIT REACHED")
        print(LINE_BREAK)

    if done:
        done = env.goal_reached()
        
    
    
    df = pd.DataFrame({'Reward': [total_reward],
                       'Steps': [steps],
                       'Network': [cols[2]],
                       'Change': [cols[3]]})
    df.to_csv('bruteforce_agent.csv', mode = 'a', index = False, header = False)

    return steps, total_reward, done

/Users/abhayraghavan/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
#run random agent
LINE_BREAK = "-"*60


def run_random_agent(env, cols, step_limit=1e6, verbose=True):
    if verbose:
        print(LINE_BREAK)
        print("STARTING EPISODE")
        print(LINE_BREAK)
        print(f"t: Reward")

    env.reset()
    total_reward = 0
    done = False
    t = 0
    a = 0

    while not done and t < step_limit:
        a = env.action_space.sample()
        _, r, done, _ = env.step(a)
        total_reward += r
        if (t+1) % 100 == 0 and verbose:
            print(f"{t}: {total_reward}")
        t += 1

    if done and verbose:
        print(LINE_BREAK)
        print("EPISODE FINISHED")
        print(LINE_BREAK)
        print(f"Total steps = {t}")
        print(f"Total reward = {total_reward}")
    elif verbose:
        print(LINE_BREAK)
        print("STEP LIMIT REACHED")
        print(LINE_BREAK)

    if done:
        done = env.goal_reached()
        
    df = pd.DataFrame({'Reward': [total_reward],
                       'Steps': [steps],
                       'Network': [cols[2]],
                       'Change': [cols[3]]})
    df.to_csv('random_agent.csv', mode = 'a', index = False, header = False)

    return t, total_reward, done

In [6]:
def run_env(filepath, change, agent):
    env = nasim.load(filepath)
    yaml = filepath[filepath.rindex("/") + 1:]

    for i in range(0, 5):
        if __name__ == "__main__":
            if (agent == 'bruteforce'):
                run_bruteforce_agent(env, ['Reward', 'Steps', yaml, change])
            if (agent == 'random'):
                run_random_agent(env, ['Reward', 'Steps', yaml, change])

In [7]:
#Edited the add subnet function to also update topology
def add_new_subnet(numOfHosts):
    curr_data['subnets'].append(numOfHosts)
    for i in range(0, len(curr_data['topology'])):
        curr_data['topology'][i].append(0)
    newRow = []
    for i in range(0, len(curr_data['topology'][0])):
        newRow.append(0)
    curr_data['topology'].append(newRow)
    
def add_new_os(os):
    if (len(curr_data['os'])) >= 1:
        curr_data['os'].append(os)
    else:
        curr_data['os'] = [os]

def add_new_process(process):
    if (len(curr_data['processes'])) >= 1:
        curr_data['processes'].append(process)
    else:
        curr_data['proceeses'] = [process]
    
def add_new_service(service):
    if (len(curr_data['services'])) >= 1:
        curr_data['services'].append(service)
    else:
        curr_data['services'] = [service]
    
def add_new_exploit(service, os, prob, cost, access):
    curr_data['exploits']['e_' + service] = {'service': service, 'os': os, 'prob': prob, 'cost': cost, 'access': access}
    
def add_new_privilege(process, os, prob, cost, access):
    curr_data['privilege_escalation']['pe_' + process] = {'process': process, 'os': os, 'prob': prob, 'cost': cost, 'access': access}
    
def add_new_firewall(x, y, services):
    address = '(' + str(x) + ', ' + str(y) + ')'
    curr_data['firewall'][address] = services
    
def add_new_sensitive_host(x, y, value):
    address = '(' + str(x) + ', ' + str(y) + ')'
    curr_data['sensitive_hosts'][address] = value
    
#Edited the add host function to also update both subnets and topology
def add_new_host_config(x, y, os, services, processes):
    address = '(' + str(x) + ', ' + str(y) + ')'
    curr_data['host_configurations'][address] = {'os': os, 'services': services, 'processes': processes}
    
    if x > len(curr_data['subnets']) - 1:
        add_new_subnet(1)
    elif y > curr_data['subnets'][x]:
        curr_data['subnets'][x] = curr_data['subnets'][x] + 1
        
    
def add_new_scan_cost(keyword, cost):
    curr_data[keyword + '_scan_cost'] = cost

In [8]:
#Edited the add subnet function to also update topology
def remove_subnet(subnetAddress):
    del curr_data['subnets'][subnetAddress]
    lenTop = len(curr_data['topology'])
    curr_data['topology'].remove(curr_data['topology'][lenTop - 1])
    lenRow = len(curr_data['topology'][0])
    for i in range(0, lenTop - 1):
        curr_data['topology'][i].remove(curr_data['topology'][i][lenRow - 1])
#def remove_topology(topologyIndex):
    #del curr_data['topology'][topologyIndex]
    
def remove_os(os):
    if os in curr_data['os']:
        curr_data['os'].remove(os)
        
def remove_process(process):
    if process in curr_data['processes']:
        curr_data['processes'].remove(process)
        
def remove_service(service):
    if service in curr_data['services']:
        curr_data['services'].remove(service)
        
def remove_exploit(exploitName):
    del curr_data['exploits'][exploitName]
    
def remove_privilege(privilegeName):
    del curr_data['privilege_escalation'][privilegeName]
    
def remove_firewall(x, y):
    address = '(' + str(x) + ', ' + str(y) + ')'
    del curr_data['firewall'][address]
    
def remove_sensitive_host(x, y):
    address = '(' + str(x) + ', ' + str(y) + ')'
    del curr_data['sensitive_hosts'][address]
    
def remove_host_config(x, y):
    address = '(' + str(x) + ', ' + str(y) + ')'
    if curr_data['subnets'][x - 1] == 1:
        remove_subnet(x - 1)
    else:
        curr_data['subnets'][x - 1] -= 1
    
    del curr_data['host_configurations'][address]
    
def remove_scan_cost(keyword):
    del curr_data[keyword + '_scan_cost']

In [9]:
def set_step_limit(stepLimit):
    curr_data['step_limit'] = stepLimit
    
#Update a YAML file with modifications that have been added or removed. 
#In this case a new file(for testing) is created to contain the updated YAML data.

def writeToYAML(filepath):
    with open(filepath, 'w') as f:
        data = yaml.dump(curr_data, f, sort_keys=False, default_flow_style=False)
        
def edit_topology(x, y, value):
    curr_data['topology'][x][y] = value

In [10]:
#An array that stores the file paths to the benchmark enviornments that are later modified
filepaths = ['../scenarios/benchmark/tiny.yaml', '../scenarios/benchmark/medium-single-site.yaml', '../scenarios/benchmark/small-linear.yaml']

In [11]:
#A sample run of the bruteforce agent on different sized networks with modifications made to them
run_env(filepaths[0], 'original', 'bruteforce')
curr_data = reset(filepaths[0])

add_new_sensitive_host(1, 0, 100)
writeToYAML(filepaths[0])
run_env(filepaths[0], 'new sensitive host', 'bruteforce')

remove_sensitive_host(1, 0)
remove_sensitive_host(2, 0)
writeToYAML(filepaths[0])
run_env(filepaths[0], 'removed sensitive host', 'bruteforce')

curr_data = reset(filepaths[1])
run_env(filepaths[1], 'original', 'bruteforce')

add_new_sensitive_host(1, 14, 100)
writeToYAML(filepaths[1])
run_env(filepaths[1], 'new sensitive host', 'bruteforce')

remove_sensitive_host(1, 14)
remove_sensitive_host(1, 3)
writeToYAML(filepaths[1])
run_env(filepaths[1], 'removed sensitive host', 'bruteforce')

curr_data = reset(filepaths[2])
run_env(filepaths[2], 'original', 'bruteforce')

add_new_sensitive_host(5, 0, 100)
writeToYAML(filepaths[2])
run_env(filepaths[2], 'new sensitive host', 'bruteforce')

remove_sensitive_host(3, 0)
writeToYAML(filepaths[2])
run_env(filepaths[2], 'removed sensitive host', 'bruteforce')



------------------------------------------------------------
STARTING EPISODE
------------------------------------------------------------
t: Reward
Total reward at step 0: -1
Total reward at step 1: -2.0
Total reward at step 2: -3.0
Total reward at step 3: -4.0
Total reward at step 4: -5.0
Total reward at step 5: -6.0
Total reward at step 6: -7.0
Total reward at step 7: -8.0
Total reward at step 8: -9.0
Total reward at step 9: -10.0
Total reward at step 10: -11.0
Total reward at step 11: -12.0
Total reward at step 12: -13.0
Total reward at step 13: -14.0
Total reward at step 14: -15.0
Total reward at step 15: -16.0
Total reward at step 16: -17.0
17: -18.0
Total reward at step 17: -18.0
Total reward at step 18: -19.0
Total reward at step 19: -20.0
Total reward at step 20: -21.0
Total reward at step 21: -22.0
Total reward at step 22: -23.0
Total reward at step 23: -24.0
Total reward at step 24: -25.0
Total reward at step 25: -26.0
Total reward at step 26: -27.0
Total reward at step 27: 

Total reward at step 14: -15.0
Total reward at step 15: -16.0
Total reward at step 16: -17.0
17: -18.0
Total reward at step 17: -18.0
Total reward at step 18: -19.0
Total reward at step 19: -20.0
Total reward at step 20: -21.0
Total reward at step 21: -22.0
Total reward at step 22: -23.0
Total reward at step 23: -24.0
Total reward at step 24: -25.0
Total reward at step 25: -26.0
Total reward at step 26: -27.0
Total reward at step 27: -28.0
Total reward at step 28: -29.0
Total reward at step 29: -30.0
Total reward at step 30: -31.0
Total reward at step 31: -32.0
Total reward at step 32: -33.0
Total reward at step 33: -34.0
Total reward at step 34: 65.0
35: 64.0
Total reward at step 35: 64.0
Total reward at step 36: 63.0
Total reward at step 37: 62.0
Total reward at step 38: 61.0
Total reward at step 39: 60.0
Total reward at step 40: 59.0
Total reward at step 41: 58.0
Total reward at step 42: 57.0
Total reward at step 43: 56.0
Total reward at step 44: 55.0
Total reward at step 45: 54.0
T

Total reward at step 36: 163.0
Total reward at step 37: 162.0
Total reward at step 38: 161.0
Total reward at step 39: 160.0
Total reward at step 40: 159.0
Total reward at step 41: 158.0
Total reward at step 42: 157.0
Total reward at step 43: 156.0
Total reward at step 44: 155.0
Total reward at step 45: 154.0
Total reward at step 46: 253.0
------------------------------------------------------------
EPISODE FINISHED
------------------------------------------------------------
Goal reached = True
Total steps = 47
Total reward = 253.0
Terminating state = {'success': True, 'value': 100.0, 'services': {}, 'os': {'linux': 1.0}, 'processes': {'tomcat': 1.0}, 'access': 2, 'discovered': {}, 'connection_error': False, 'permission_error': False, 'newly_discovered': {}}
{'success': True, 'value': 0, 'services': {}, 'os': {'linux': 1.0}, 'processes': {}, 'access': {}, 'discovered': {}, 'connection_error': False, 'permission_error': False, 'newly_discovered': {}}
------------------------------------

Total reward at step 6: -7.0
Total reward at step 7: -8.0
Total reward at step 8: -9.0
Total reward at step 9: -10.0
Total reward at step 10: -11.0
Total reward at step 11: -12.0
Total reward at step 12: -13.0
Total reward at step 13: -14.0
Total reward at step 14: -15.0
Total reward at step 15: -16.0
Total reward at step 16: -17.0
17: -18.0
Total reward at step 17: -18.0
Total reward at step 18: -19.0
Total reward at step 19: -20.0
Total reward at step 20: -21.0
Total reward at step 21: -22.0
Total reward at step 22: -23.0
Total reward at step 23: -24.0
Total reward at step 24: -25.0
Total reward at step 25: -26.0
Total reward at step 26: -27.0
Total reward at step 27: -28.0
Total reward at step 28: -29.0
Total reward at step 29: -30.0
Total reward at step 30: -31.0
Total reward at step 31: -32.0
Total reward at step 32: -33.0
Total reward at step 33: -34.0
Total reward at step 34: -35.0
35: -36.0
Total reward at step 35: -36.0
Total reward at step 36: -37.0
Total reward at step 37: -

Total reward at step 4: -5.0
Total reward at step 5: -6.0
Total reward at step 6: -7.0
Total reward at step 7: -8.0
Total reward at step 8: -9.0
Total reward at step 9: -10.0
Total reward at step 10: -11.0
Total reward at step 11: -12.0
Total reward at step 12: -13.0
Total reward at step 13: -14.0
Total reward at step 14: -15.0
Total reward at step 15: -16.0
Total reward at step 16: -17.0
17: -18.0
Total reward at step 17: -18.0
Total reward at step 18: -19.0
Total reward at step 19: -20.0
Total reward at step 20: -21.0
Total reward at step 21: -22.0
Total reward at step 22: -23.0
Total reward at step 23: -24.0
Total reward at step 24: -25.0
Total reward at step 25: -26.0
Total reward at step 26: -27.0
Total reward at step 27: -28.0
Total reward at step 28: -29.0
Total reward at step 29: -30.0
Total reward at step 30: -31.0
Total reward at step 31: -32.0
Total reward at step 32: -33.0
Total reward at step 33: -34.0
Total reward at step 34: -35.0
35: -36.0
Total reward at step 35: -36.0

...

------------------------------------------------------------
STARTING EPISODE
------------------------------------------------------------
t: Reward
Total reward at step 0: -1
Total reward at step 1: -2.0
Total reward at step 2: -3.0
Total reward at step 3: -6.0
Total reward at step 4: -7.0
Total reward at step 5: -9.0
Total reward at step 6: -10.0
Total reward at step 7: -11.0
Total reward at step 8: -12.0
Total reward at step 9: -13.0
Total reward at step 10: -14.0
Total reward at step 11: -15.0
Total reward at step 12: -18.0
Total reward at step 13: -19.0
Total reward at step 14: -21.0
Total reward at step 15: -22.0
Total reward at step 16: -23.0
Total reward at step 17: -24.0
Total reward at step 18: -25.0
Total reward at step 19: -26.0
Total reward at step 20: -27.0
Total reward at step 21: -30.0
Total reward at step 22: -31.0
Total reward at step 23: -33.0
Total reward at step 24: -34.0
Total reward at step 25: -35.0
Total reward at step 26: -36.0
Total reward at step 27: -37.0
T

Total reward at step 251: -336.0
Total reward at step 252: -337.0
Total reward at step 253: -338.0
Total reward at step 254: -339.0
Total reward at step 255: -342.0
Total reward at step 256: -343.0
Total reward at step 257: -345.0
Total reward at step 258: -346.0
Total reward at step 259: -247.0
Total reward at step 260: -248.0
Total reward at step 261: -249.0
Total reward at step 262: -250.0
Total reward at step 263: -251.0
Total reward at step 264: -254.0
Total reward at step 265: -255.0
Total reward at step 266: -257.0
Total reward at step 267: -258.0
Total reward at step 268: -259.0
Total reward at step 269: -260.0
Total reward at step 270: -261.0
Total reward at step 271: -262.0
Total reward at step 272: -263.0
Total reward at step 273: -266.0
Total reward at step 274: -267.0
Total reward at step 275: -269.0
Total reward at step 276: -270.0
Total reward at step 277: -271.0
Total reward at step 278: -272.0
Total reward at step 279: -273.0
Total reward at step 280: -274.0
Total rewa

Total reward at step 502: -571.0
503: -572.0
Total reward at step 503: -572.0
Total reward at step 504: -573.0
Total reward at step 505: -574.0
Total reward at step 506: -575.0
Total reward at step 507: -578.0
Total reward at step 508: -579.0
Total reward at step 509: -581.0
Total reward at step 510: -582.0
Total reward at step 511: -583.0
Total reward at step 512: -584.0
Total reward at step 513: -585.0
Total reward at step 514: -586.0
Total reward at step 515: -587.0
Total reward at step 516: -590.0
Total reward at step 517: -591.0
Total reward at step 518: -593.0
Total reward at step 519: -594.0
Total reward at step 520: -595.0
Total reward at step 521: -596.0
Total reward at step 522: -597.0
Total reward at step 523: -598.0
Total reward at step 524: -599.0
Total reward at step 525: -602.0
Total reward at step 526: -603.0
Total reward at step 527: -605.0
Total reward at step 528: -606.0
Total reward at step 529: -607.0
Total reward at step 530: -608.0
Total reward at step 531: -609.

Total reward at step 750: -902.0
Total reward at step 751: -903.0
Total reward at step 752: -905.0
Total reward at step 753: -906.0
Total reward at step 754: -907.0
Total reward at step 755: -908.0
Total reward at step 756: -909.0
Total reward at step 757: -910.0
Total reward at step 758: -911.0
Total reward at step 759: -914.0
Total reward at step 760: -915.0
Total reward at step 761: -917.0
Total reward at step 762: -918.0
Total reward at step 763: -919.0
Total reward at step 764: -920.0
Total reward at step 765: -921.0
Total reward at step 766: -922.0
Total reward at step 767: -923.0
Total reward at step 768: -926.0
Total reward at step 769: -927.0
Total reward at step 770: -929.0
Total reward at step 771: -930.0
Total reward at step 772: -931.0
Total reward at step 773: -932.0
Total reward at step 774: -933.0
Total reward at step 775: -934.0
Total reward at step 776: -935.0
Total reward at step 777: -938.0
Total reward at step 778: -939.0
Total reward at step 779: -941.0
Total rewa

Total reward at step 994: -1227.0
Total reward at step 995: -1229.0
Total reward at step 996: -1230.0
Total reward at step 997: -1231.0
Total reward at step 998: -1232.0
Total reward at step 999: -1233.0
------------------------------------------------------------
EPISODE FINISHED
------------------------------------------------------------
Goal reached = False
Total steps = 1000
Total reward = -1233.0
Terminating state = {'success': True, 'value': 0, 'services': {}, 'os': {'linux': 1.0, 'windows': 0.0}, 'processes': {}, 'access': {}, 'discovered': {}, 'connection_error': False, 'permission_error': False, 'newly_discovered': {}}
{'success': True, 'value': 0, 'services': {}, 'os': {'linux': 1.0}, 'processes': {}, 'access': {}, 'discovered': {}, 'connection_error': False, 'permission_error': False, 'newly_discovered': {}}


In [ ]:
import csv
import pandas as pd
df = pd.read_csv('bruteforce_agent_sensitive_host.csv')